## Research Question
Can we calculate the probability that a player is selected MVP for a season based on their stats for the season? 

## Data Cleaning & Collection
The content below describes the process for collecting the data from the NBA site.



In [104]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import duckdb
import numpy as np
import time

**Player Stat Data Collection** 
In other file

**TO DOs & QUESTIONS**
Need to save these files as .csv then show the cleaning process with code here
Explain collection process with text and refer the TA grader to github to see the process of collecting the data and converting it into "raw" csv files that could then be merged/manipulated and cleaned
Should we clean the data (i.e. get rid of the data we don't want to use?)

Here I wanted to update the team names so they are consistent with other data frames. We should consider whether we should use the abbreviations or the full names?

In [105]:
#read csv file "player_stats.csv"
player_stats_df = pd.read_csv('player_stats.csv')
player_stats_df
team_name_mapping = {
    'ATL': 'Atlanta Hawks',
    'BOS': 'Boston Celtics',
    'BKN': 'Brooklyn Nets',
    'CHA': 'Charlotte Hornets',
    'CHI': 'Chicago Bulls',
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'LA Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans',
    'NYK': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers',
    'PHX': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SAS': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards',
}

player_stats_df['Team_Name'] = player_stats_df['TEAM'].map(team_name_mapping)

#Check to see that the head and tail of the data is popoulating as expected with updated team names
player_stats_df



,Unnamed: 0,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,...,DREB,REB,AST,STL,BLK,TOV,PTS,EFF,Year,Team_Name
0,0,1503,1,Tracy McGrady,1610612753,ORL,67,39.9,9.7,23.4,...,4.6,6.0,5.5,1.4,0.6,2.7,28.0,23.7,2003-04,Orlando Magic
1,1,978,2,Peja Stojakovic,1610612758,SAC,81,40.3,8.2,17.1,...,5.1,6.3,2.1,1.3,0.2,1.9,24.2,23.0,2003-04,Sacramento Kings
2,2,708,3,Kevin Garnett,1610612750,MIN,82,39.4,9.8,19.6,...,10.9,13.9,5.0,1.5,2.2,2.6,24.2,33.1,2003-04,Minnesota Timberwolves
3,3,977,4,Kobe Bryant,1610612747,LAL,65,37.7,7.9,18.1,...,3.9,5.5,5.1,1.7,0.4,2.6,24.0,22.7,2003-04,Los Angeles Lakers
4,4,1718,5,Paul Pierce,1610612738,BOS,80,38.8,7.5,18.7,...,5.7,6.5,5.1,1.6,0.7,3.8,23.0,20.5,2003-04,Boston Celtics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4201,4201,200782,241,P.J. Tucker,1610612755,PHI,75,25.6,1.3,3.0,...,2.7,3.9,0.8,0.5,0.2,0.6,3.5,6.6,2022-23,Philadelphia 76ers
4202,4202,1630540,242,Miles McBride,1610612752,NYK,64,11.9,1.2,3.4,...,0.6,0.8,1.1,0.6,0.1,0.4,3.5,3.3,2022-23,New York Knicks
4203,4203,1630264,243,Anthony Gill,1610612764,WAS,59,10.6,1.2,2.2,...,1.1,1.7,0.6,0.1,0.2,0.3,3.3,4.3,2022-23,Washington Wizards
4204,4204,1631132,244,Christian Koloko,1610612761,TOR,58,13.8,1.2,2.6,...,1.5,2.9,0.5,0.4,1.0,0.3,3.1,5.9,2022-23,Toronto Raptors


Again - which should we use...abbreviations or full name? Make it clean and consistent

In [106]:

#Mapping in order to standardize the team names across data frames
# team_name_mapping = {
#     'Atlanta': 'Atlanta Hawks',
#     'Boston': 'Boston Celtics',
#     'Brooklyn': 'Brooklyn Nets',
#     'Charlotte': 'Charlotte Hornets',
#     'Chicago': 'Chicago Bulls',
#     'Cleveland': 'Cleveland Cavaliers',
#     'Dallas': 'Dallas Mavericks',
#     'Denver': 'Denver Nuggets',
#     'Detroit': 'Detroit Pistons',
#     'Golden State': 'Golden State Warriors',
#     'Houston': 'Houston Rockets',
#     'Indiana': 'Indiana Pacers',
#     'LA Clippers': 'LA Clippers',
#     'LA Lakers': 'LA Lakers',
#     'Memphis': 'Memphis Grizzlies',
#     'Miami': 'Miami Heat',
#     'Milwaukee': 'Milwaukee Bucks',
#     'Minnesota': 'Minnesota Timberwolves',
#     'New Orleans': 'New Orleans Pelicans',
#     'New York': 'New York Knicks',
#     'Okla City': 'Oklahoma City Thunder',
#     'Orlando': 'Orlando Magic',
#     'Philadelphia': 'Philadelphia 76ers',
#     'Phoenix': 'Phoenix Suns',
#     'Portland': 'Portland Trail Blazers',
#     'Sacramento': 'Sacramento Kings',
#     'San Antonio': 'San Antonio Spurs',
#     'Toronto': 'Toronto Raptors',
#     'Utah': 'Utah Jazz',
#     'Washington': 'Washington Wizards',
# }

# team_stats_df['Team'] = team_stats_df['Team'].map(team_name_mapping)

# #Add rank starting at index 1 
# team_stats_df.insert(0, 'RANK', range(1, len(team_stats_df) + 1))

# team_stats_df


**TO DO**
Collect data about: 
Gather data about the # of wins and losses of the team
Calculate percent win rate
Calculate total number of games played in regular season (should be 82)

**Cleaning data about historical MVPs**


In [109]:

mvp_df = pd.read_csv('mvp_historical.csv')
mvp_df


mvp_team_df = duckdb.sql("""
SELECT mvp_df.Year, mvp_df.MVP_Name, player_stats_df.Team_Name, player_stats_df.TEAM
FROM mvp_df
LEFT JOIN player_stats_df ON mvp_df.MVP_Name = player_stats_df.PLAYER
""").df()

mvp_team_df

#why are team names the abbreviations? i thought i mapped/modified them?



,Year,MVP_Name,Team_Name,TEAM
0,2003-04,Kevin Garnett,Minnesota Timberwolves,MIN
1,2007-08,Kobe Bryant,Los Angeles Lakers,LAL
2,2006-07,Dirk Nowitzki,Dallas Mavericks,DAL
3,2012-13,LeBron James,Cleveland Cavaliers,CLE
4,2005-06,Steve Nash,Dallas Mavericks,DAL
...,...,...,...,...
201,2009-10,LeBron James,Los Angeles Lakers,LAL
202,2008-09,LeBron James,Cleveland Cavaliers,CLE
203,2008-09,LeBron James,Cleveland Cavaliers,CLE
204,2008-09,LeBron James,Cleveland Cavaliers,CLE


**Collecting Data About Players Rank Within Their Team**
TO DO: Get the player ranking within their team?
Describe process here


In [ ]:

# #  Merge dataframes on 'Year' and 'TEAM' columns
# merged_df = player_stats_df.merge(team_stats_df, on=['Year', 'Team'], how='left')

# # Create a new column to calculate player's rank within their team for each season
# #TO DO

# # Sort the merged dataframe by 'Year', 'Team Name'
# merged_df = merged_df.sort_values(by=['Year', 'Team'])
